In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [2]:
data = "../WFs_reliability/"

In [10]:
loader = PyPDFDirectoryLoader(data)

docs_before_split = loader.load()

text_full = ""
for doc in docs_before_split:
    text_full += doc.page_content

print(text_full)

 1 
 
 
 
风力发电设备可靠性评价规程（试行）  
 
       2 
目           次  
  前言………………………………………………………………………………. 1   范围………………………………………………………………………………… 2   基本要求 3   状态划分 4   术语和定义 5   状态转变时间界线和时间记录的规定 6   评价指标 7   状态填报的规定 8   风电机组评价指标 9   风电场评价指标 10  基础数据注册 11  事件数据填写规定 12  统计评价报告 附录 A(资料性附录)风电设备可靠性统计状态中、英文对照表 附录 B(资料性附录)风电设备可靠性指标中、英文对照表 参考文献  3前           言  
  
    风力发电设备可靠性评价是电力可靠性管理的重要组成部分。 本标准是在《发电设备可靠性评价规程》的基础上，结合风力发电设备的具体情况编写的。本标准经过多次专题会议讨论，征询有关专家的意见，并将专家的多数意见、建议吸收和补充进来。 
本规程详细规定了风力发电机组 和风电场可靠性统计评价办法和
评价指标。 
本规程还参考了以下标准： GB/T 3187-1994，可靠性、维修性术语； IEC271,可靠性的基本名词、定义和有关数据； 本规程（试行）的附录A、附录B 为资料性附录。 本规程（试行）由电力可靠性标 准化技术委员会提出、归口并负
责解释。 
本规程（试行）起草单位：中国 电力企业联合会电力可靠性管理
中心。 
  41  范围  
1.1 本规程规定了风力发电设备可靠性的统计办法和评价指标。 适
用于我国境内的所有风力发电企业。  
1.2 风力发电设备的可靠性统计评价包括风电机组的可靠性统计
评价和风电场的可靠性统计评价两部分。  
1.3 风电机组的可靠性统计评价范围以风电机组出口主开关为界，
包括风轮、传动变速系统、发电机系统、液压系统、偏航系统、控制
系统、变桨系统、通讯系统以及相应的辅助系统。  
1.4 风电场的可靠性统计评价范围包括风电场内的所有发电设备，
除了风电机组外，还包括箱变、汇流 线路、主变等，及其相应的附属、
辅助设备，公用系统和设施。  
2  基本要求  
2.1 本规程中指标评价所要求的各种基础数据报告，必须尊重科
学、事实求是、严肃认真、全面而